In [2]:
import open3d as o3d
import numpy as np
from scipy.stats import wasserstein_distance


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
def get_outs_dataname(dirname: str, dataIndex: str):
  return f"./outs/{dirname}/outputs/{dataIndex}_0.0.ply"

# 生成文件名的函数
def generate_filename_prefixes(start, end, step):
    filenames = []
    for i in range(start, end + 1, step):
        # 用zfill补全前面的0，8表示总共8位数字
        filename = f"{str(i).zfill(8)}"
        filenames.append(filename)
    return filenames


# get the mean chamfer distance between two point clouds
def calculate_average_point_cloud_distance(pc1_path: str, pc2_path: str):
    """
    Calculate the average distance between two point clouds.
    
    Args:
    pc1_path (str): Path to the first point cloud
    pc2_path (str): Path to the second point cloud
    
    Returns:
    float: Average distance between the two point clouds
    """
    # Load the point clouds
    pc1 = o3d.io.read_point_cloud(pc1_path)
    pc2 = o3d.io.read_point_cloud(pc2_path)
    
    # Compute the distance between the two point clouds using cd (Chamfer Distance)
    distance = pc1.compute_point_cloud_distance(pc2)

    # average the distance
    return np.mean(distance)

def calculate_average_point_cloud_distance(pc1: o3d.geometry.PointCloud, pc2: o3d.geometry.PointCloud):
    """
    Calculate the average distance between two point clouds.
    
    Args:
    pc1 (o3d.geometry.PointCloud): The first point cloud
    pc2 (o3d.geometry.PointCloud): The second point cloud
    
    Returns:
    float: Average distance between the two point clouds
    """
    # Compute the distance between the two point clouds using cd (Chamfer Distance)
    distance = pc1.compute_point_cloud_distance(pc2)

    # average the distance
    return np.mean(distance)


def compute_emd(pc1_path, pc2_path):
    """
    Compute the Earth Mover's Distance between two point clouds.
    
    Args:
    pc1_path (str): Path to the first point cloud file.
    pc2_path (str): Path to the second point cloud file.
    
    Returns:
    float: The computed EMD between the two point clouds.
    """
    # Read the point clouds
    pc1 = o3d.io.read_point_cloud(pc1_path)
    pc2 = o3d.io.read_point_cloud(pc2_path)


    # print the size of the point clouds
    print("Size of point cloud 1: ", len(pc1.points))
    print("Size of point cloud 2: ", len(pc2.points))
    

    # Convert point clouds to numpy arrays
    pc1_np = np.asarray(pc1.points)
    pc2_np = np.asarray(pc2.points)

    # Ensure both point clouds have the same number of points (required for EMD)
    min_points = min(pc1_np.shape[0], pc2_np.shape[0])
    pc1_np = pc1_np[:min_points]
    pc2_np = pc2_np[:min_points]

    # Compute EMD for each dimension
    emd_x = wasserstein_distance(pc1_np[:, 0], pc2_np[:, 0])
    emd_y = wasserstein_distance(pc1_np[:, 1], pc2_np[:, 1])
    emd_z = wasserstein_distance(pc1_np[:, 2], pc2_np[:, 2])

    # Compute the total EMD as the sum of the EMDs for each dimension
    total_emd = emd_x + emd_y + emd_z

    return total_emd

def compute_emd(pc1: o3d.geometry.PointCloud, pc2: o3d.geometry.PointCloud):
    """
    Compute the Earth Mover's Distance between two point clouds.
    
    Args:
    pc1 (o3d.geometry.PointCloud): The first point cloud.
    pc2 (o3d.geometry.PointCloud): The second point cloud.
    
    Returns:
    float: The computed EMD between the two point clouds.
    """
    # Convert point clouds to numpy arrays
    pc1_np = np.asarray(pc1.points)
    pc2_np = np.asarray(pc2.points)

    # Ensure both point clouds have the same number of points (required for EMD)
    min_points = min(pc1_np.shape[0], pc2_np.shape[0])
    pc1_np = pc1_np[:min_points]
    pc2_np = pc2_np[:min_points]

    # Compute EMD for each dimension
    emd_x = wasserstein_distance(pc1_np[:, 0], pc2_np[:, 0])
    emd_y = wasserstein_distance(pc1_np[:, 1], pc2_np[:, 1])
    emd_z = wasserstein_distance(pc1_np[:, 2], pc2_np[:, 2])

    # Compute the total EMD as the sum of the EMDs for each dimension
    total_emd = emd_x + emd_y + emd_z

    return total_emd

In [ ]:
original_point_cloud = o3d.io.read_point_cloud("./data/gargoyle.ply")
# visualize the point cloud
# o3d.visualization.draw_geometries([original_point_cloud])
# get the number of points in the point cloud
original_number_of_points = np.asarray(original_point_cloud.points).shape[0]
print(f"Original number of points: {original_number_of_points}")


target_pc = o3d.io.read_triangle_mesh(get_outs_dataname("gargoyle", "00037500"))
target_pc = target_pc.sample_points_uniformly(number_of_points=original_number_of_points)


test_array = generate_filename_prefixes(7500, 37500, 2500)


# show the point cloud
# o3d.visualization.draw_geometries([test_point_cloud])

# chamfer distance
for i in test_array:
    test_point_cloud = o3d.io.read_triangle_mesh(get_outs_dataname("gargoyle", i))
    test_point_cloud = test_point_cloud.sample_points_uniformly(number_of_points=original_number_of_points)
    print(f"Average distance between original and {i}: {calculate_average_point_cloud_distance(target_pc, test_point_cloud)}")

for i in test_array:
    test_point_cloud = o3d.io.read_triangle_mesh(get_outs_dataname("gargoyle", i))
    test_point_cloud = test_point_cloud.sample_points_uniformly(number_of_points=original_number_of_points)
    print(f"EMD between original and {i}: {compute_emd(target_pc, test_point_cloud)}")